In [25]:
import torchvision.models as models
import torch
from ptflops import get_model_complexity_info
from timm.models import create_model
import models.AIM



model = create_model(
    'aim_adapter_vit_base_patch16_224'
    # pretrained=False,
    # num_classes=300,
    # all_frames=16 * args.num_segments,
    # tubelet_size=args.tubelet_size,
    # drop_rate=0.3,
    # drop_path_rate=args.drop_path,
    # attn_drop_rate=args.attn_drop_rate,
    # drop_block_rate=None,
    # use_mean_pooling=args.use_mean_pooling,
    # init_scale=args.init_scale,
    # fusion_method=args.fusion_method,
#   head_drop_rate=args.head_drop
)
# (3, 8, 568, 320)
macs, params = get_model_complexity_info(model, (3, 16, 224, 224), as_strings=True,
                                        print_per_layer_stat=True, verbose=True)
print('rrr',macs)
print(params)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

STCrossTransformer(
  103.67 M, 99.803% Params, 405.3 GMac, 99.995% MACs, 
  (clip_conv1): Conv2d(589.82 k, 0.568% Params, 1.85 GMac, 0.456% MACs, 3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
  (clip_ln_pre): LayerNorm(0, 0.000% Params, 0.0 Mac, 0.000% MACs, (768,), eps=1e-05, elementwise_affine=True)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      8.56 M, 8.245% Params, 33.62 GMac, 8.295% MACs, 
      (act): GELU(0, 0.000% Params, 0.0 Mac, 0.000% MACs, approximate='none')
      (clip_ln_1): LayerNorm(0, 0.000% Params, 0.0 Mac, 0.000% MACs, (768,), eps=1e-05, elementwise_affine=True)
      (WT_Adapter): Adapter(
        295.87 k, 0.285% Params, 0.0 Mac, 0.000% MACs, 
        (act): GELU(0, 0.000% Params, 0.0 Mac, 0.000% MACs, approximate='none')
        (D_fc1): Linear(147.65 k, 0.142% Params, 0.0 Mac, 0.000% MACs, in_features=768, out_features=192, bias=True)
        (D_fc2): Linear(148.22 k, 0.143% Params, 0.0 Mac, 0.000% MACs, in_features=192, out_features=768, b

In [6]:
class argpar:
    def __init__(self):
        self.vmae_model = 'aim_adapter_vit_base_patch16_224'
        self.nb_classes = '2'
        
        
args = argpar()


print(args.vmae_model)
print(args.nb_classes)


aim_adapter_vit_base_patch16_224
2


In [27]:
from torch.profiler import profile, record_function, ProfilerActivity
import torchvision.models as models
import torch
from ptflops import get_model_complexity_info
from timm.models import create_model
import models.AIM



model = create_model(
    'aim_adapter_vit_base_patch16_224'
    # pretrained=False,
    # num_classes=300,
    # all_frames=16 * args.num_segments,
    # tubelet_size=args.tubelet_size,
    # drop_rate=0.3,
    # drop_path_rate=args.drop_path,
    # attn_drop_rate=args.attn_drop_rate,
    # drop_block_rate=None,
    # use_mean_pooling=args.use_mean_pooling,
    # init_scale=args.init_scale,
    # fusion_method=args.fusion_method,
#   head_drop_rate=args.head_drop
)
# (3, 8, 568, 320)
inputs = torch.rand(5, 3, 16, 224, 224)
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)
        
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# macs, params = get_model_complexity_info(model, (3, 16, 224, 224), as_strings=True,
#                                         print_per_layer_stat=True, verbose=True)
# print('rrr',macs)
# print(params)
# print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
# print('{:<30}  {:<8}'.format('Number of parameters: ', params))

STAGE:2023-12-26 15:05:40 3986445:3986445 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             model_inference         4.66%     609.153ms       100.00%       13.080s       13.080s             1  
                                aten::linear         0.02%       2.355ms        74.24%        9.711s      66.514ms           146  
                                 aten::addmm        69.95%        9.150s        74.20%        9.705s      66.472ms           146  
                                 aten::copy_         8.68%        1.135s         8.68%        1.135s       3.569ms           318  
          aten::scaled_dot_product_attention         0.00%      84.000us         7.

STAGE:2023-12-26 15:05:53 3986445:3986445 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-12-26 15:05:53 3986445:3986445 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [28]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             model_inference         4.66%     609.153ms       100.00%       13.080s       13.080s             1                                                                                []  
                                 aten::addmm        23.59%        3.085s        25.03%        3.274s     136.413ms            24                    

In [29]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             model_inference         4.66%     609.153ms       100.00%       13.080s       13.080s             1  
                               aten::permute         0.00%     570.000us         0.00%     644.000us      11.926us            54  
                            aten::as_strided         0.01%     681.000us         0.01%     681.000us       0.988us           689  
                               aten::reshape         0.01%       1.402ms         0.77%     100.997ms     668.854us           151  
                                 aten::clone         0.01%       1.216ms         4.

In [30]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             model_inference         4.66%     609.153ms       100.00%       13.080s       13.080s             1  
                               aten::permute         0.00%     570.000us         0.00%     644.000us      11.926us            54  
                            aten::as_strided         0.01%     681.000us         0.01%     681.000us       0.988us           689  
                               aten::reshape         0.01%       1.402ms         0.77%     100.997ms     668.854us           151  
                                 aten::clone         0.01%       1.216ms         4.

In [3]:
import torch
clip_finetune = '/data/datasets/Epickitchens100_clips/ViT-B-16.pt'
clip_checkpoint = torch.jit.load(clip_finetune, map_location='cpu')

print("Load CLIP ckpt from %s" % clip_finetune)

Load CLIP ckpt from /data/datasets/Epickitchens100_clips/ViT-B-16.pt


In [4]:
checkpoint_clip = clip_checkpoint.visual.state_dict()

In [5]:
print(checkpoint_clip.keys())

odict_keys(['class_embedding', 'positional_embedding', 'proj', 'conv1.weight', 'ln_pre.weight', 'ln_pre.bias', 'transformer.resblocks.0.attn.in_proj_weight', 'transformer.resblocks.0.attn.in_proj_bias', 'transformer.resblocks.0.attn.out_proj.weight', 'transformer.resblocks.0.attn.out_proj.bias', 'transformer.resblocks.0.ln_1.weight', 'transformer.resblocks.0.ln_1.bias', 'transformer.resblocks.0.mlp.c_fc.weight', 'transformer.resblocks.0.mlp.c_fc.bias', 'transformer.resblocks.0.mlp.c_proj.weight', 'transformer.resblocks.0.mlp.c_proj.bias', 'transformer.resblocks.0.ln_2.weight', 'transformer.resblocks.0.ln_2.bias', 'transformer.resblocks.1.attn.in_proj_weight', 'transformer.resblocks.1.attn.in_proj_bias', 'transformer.resblocks.1.attn.out_proj.weight', 'transformer.resblocks.1.attn.out_proj.bias', 'transformer.resblocks.1.ln_1.weight', 'transformer.resblocks.1.ln_1.bias', 'transformer.resblocks.1.mlp.c_fc.weight', 'transformer.resblocks.1.mlp.c_fc.bias', 'transformer.resblocks.1.mlp.c_pr

In [8]:
model_key = 'model|module'

for model_key in model_key.split('|'):
    if model_key in checkpoint_clip:
        checkpoint_model = checkpoint_clip[model_key]
        print("Load state_dict by model_key = %s" % model_key)
        break
    print(model_key)

    
#     if model_key in checkpoint:
#         checkpoint_model = checkpoint[model_key]
#         print("Load state_dict by model_key = %s" % model_key)
#         break
# if checkpoint_model is None:
#     checkpoint_model = checkpoint
# state_dict = model.state_dict()
# for k in ['head.weight', 'head.bias']:
#     if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
#         print(f"Removing key {k} from pretrained checkpoint")
#         del checkpoint_model[k]

model
module


In [9]:
clip_all_keys = list(checkpoint_clip.keys())
new_dict = OrderedDict()
for key in clip_all_keys:
        if key.startswith('transformer.'):
            if key[23] == '.':
                new_dict['blocks.'+ key[22] + '.clip_' + key[24:]] = checkpoint_clip[key]
            else : # layer10 ~ 11 process
                new_dict['blocks.'+ key[22:24] + '.clip_' + key[25:]] = checkpoint_clip[key]
        else:
            new_dict['clip_' + key] = checkpoint_clip[key]

['class_embedding', 'positional_embedding', 'proj', 'conv1.weight', 'ln_pre.weight', 'ln_pre.bias', 'transformer.resblocks.0.attn.in_proj_weight', 'transformer.resblocks.0.attn.in_proj_bias', 'transformer.resblocks.0.attn.out_proj.weight', 'transformer.resblocks.0.attn.out_proj.bias', 'transformer.resblocks.0.ln_1.weight', 'transformer.resblocks.0.ln_1.bias', 'transformer.resblocks.0.mlp.c_fc.weight', 'transformer.resblocks.0.mlp.c_fc.bias', 'transformer.resblocks.0.mlp.c_proj.weight', 'transformer.resblocks.0.mlp.c_proj.bias', 'transformer.resblocks.0.ln_2.weight', 'transformer.resblocks.0.ln_2.bias', 'transformer.resblocks.1.attn.in_proj_weight', 'transformer.resblocks.1.attn.in_proj_bias', 'transformer.resblocks.1.attn.out_proj.weight', 'transformer.resblocks.1.attn.out_proj.bias', 'transformer.resblocks.1.ln_1.weight', 'transformer.resblocks.1.ln_1.bias', 'transformer.resblocks.1.mlp.c_fc.weight', 'transformer.resblocks.1.mlp.c_fc.bias', 'transformer.resblocks.1.mlp.c_proj.weight',